In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My Drive/AtoZ/Deep_Learning_A_Z/Boltzmann_Machines

/content/drive/My Drive/AtoZ/Deep_Learning_A_Z/Boltzmann_Machines


In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
#Importing dataset
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding = 'latin-1')

In [0]:
#prepare training set and test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t', header=None)
training_set = np.array(training_set, dtype='int')


test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t', header=None)
test_set = np.array(test_set, dtype='int')

In [0]:
#insert number of users and movies
combined = np.append(training_set, test_set, axis=0)
nb_users = len(np.unique(combined[:,0])) #943
nb_movies = len(np.unique(combined[:,1])) #1682

In [0]:
def convert(data):
  new_data=[]
  for id_users in range(1, nb_users + 1):
    id_movies = data[:,1][data[:,0]==id_users]
    #print(id_movies)
    id_ratings = data[:, 2][data[:,0]==id_users]
    #print(id_ratings)
    ratings = np.zeros(nb_movies)
    ratings[id_movies-1] = id_ratings
    #print(ratings)
    new_data.append(list(ratings))
  return new_data

In [0]:
training_set = convert(training_set)
test_set = convert(test_set)


In [0]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies,20)
    self.fc2 = nn.Linear(20,10)
    self.fc3 = nn.Linear(10,20)
    self.fc4 = nn.Linear(20,nb_movies)
    self.activation = nn.Sigmoid()
    
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

In [0]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [0]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s =0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae.forward(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.item()*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch : '+str(epoch)+' loss : '+str(train_loss/s))

epoch : 1 loss : 1.0160554649088045
epoch : 2 loss : 1.016833106133363
epoch : 3 loss : 1.0172460932074407
epoch : 4 loss : 1.0167354864320395
epoch : 5 loss : 1.0166421261459593
epoch : 6 loss : 1.0162578264192623
epoch : 7 loss : 1.016516851879663
epoch : 8 loss : 1.0163199665873832
epoch : 9 loss : 1.0161654467961951
epoch : 10 loss : 1.01577897300205
epoch : 11 loss : 1.0160097235417567
epoch : 12 loss : 1.0156568439617368
epoch : 13 loss : 1.0159224498581603
epoch : 14 loss : 1.0155650415989113
epoch : 15 loss : 1.0156849960867371
epoch : 16 loss : 1.0150757578776286
epoch : 17 loss : 1.0148304967707034
epoch : 18 loss : 1.013338923052488
epoch : 19 loss : 1.0125138118478132
epoch : 20 loss : 1.009173456184543
epoch : 21 loss : 1.0098087824179813
epoch : 22 loss : 1.0057050191409513
epoch : 23 loss : 1.0048892385757124
epoch : 24 loss : 1.0014628098111376
epoch : 25 loss : 0.9984275849611918
epoch : 26 loss : 0.9976638504656802
epoch : 27 loss : 0.9958007021809756
epoch : 28 loss 